In [ ]:
# Extract results

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import time
import json

import pandas as pd

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer, SharedVehicleLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_layer_from_roads
from mnms.vehicles.veh_type import Tram, Metro, Bus, Bike
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.mobility_service.vehicle_sharing import VehicleSharingMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, load_odlayer, save_transit_links
from mnms.tools.render import draw_roads, draw_line, draw_odlayer, draw_path, draw_veh_activity
#from mnms.tools.geometry import points_in_polygon, get_bounding_box
from mnms.time import Time

In [ ]:
### Parameters

# Files and directories
f = open('params.json')
params = json.load(f)

current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

#coord_csv_filepath = indir + 'KV1_GVB_2609_2/Csv/POINT.csv' # file with coordinates of the network
#amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only
#amsterdam_json_filepath_pt_transit = indir + "network_pt_transit.json"
#transit_path = indir + "transit.json"

In [ ]:
### Load network

mmgraph_pt = load_graph(indir + params["fn_network"])

### Load demand

df_dmd = pd.read_csv(indir + params['fn_demand'], sep=";")

### Load odlayer
odlayer = load_odlayer(indir + params["fn_odlayer"])
x_od = []
y_od = []
for i in odlayer.origins.items():
    x = i[1][0]
    y = i[1][1]
    x_od.append(x)
    y_od.append(y)

In [ ]:
## Load results

df_path = pd.read_csv(outdir+'path.csv', sep=";")
df_users = pd.read_csv(outdir+'users.csv', sep=";")
df_flow = pd.read_csv(outdir+'flow.csv', sep=";")

# Plot network

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

draw_roads(ax, mmgraph_pt.roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

# Params of the visualization
colors = {'BUS': 'green', 'METRO': 'red', 'TRAM': 'skyblue'}

for layer in mmgraph_pt.layers.values():
    if type(layer) == PublicTransportLayer:
        for name, line in layer.lines.items():
                draw_line(ax, mmgraph_pt, line, color=colors[name[:name.find('_')]], 
                          linkwidth=0.4, nodesize=1, line_label=None, label_size=1, alpha=1., stopmarkeredgewidth=0.1)

legend = [Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='skyblue', linewidth=5,
            label='Tram line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='red', linewidth=5,
            label='Metro line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='green', linewidth=5,
            label='Bus line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=0, markeredgecolor='grey', markerfacecolor='grey', color='grey', linewidth=1,
            label='Roads')]

legend = plt.legend(handles=legend)
#plt.savefig(params['figdir']+'network.pdf')
#plt.xlim([627000,628000])
#plt.ylim([5.8095e6, 5.8105e6])

# Traffic dynamics

In [ ]:
df_flow

In [ ]:
df_cars = df_flow[df_flow.VEHICLE_TYPE=="CAR"]

In [ ]:
plt.plot

fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('speed', color=color)
ax1.plot(df_cars.SPEED.values, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('accumulation', color=color)  # we already handled the x-label with ax1
ax2.plot(df_cars.ACCUMULATION.values, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

## Modes shares

In [ ]:
def car_in_modes(modes):
    if pd.isna(modes):
        return False
    else:
        return 'CAR' in modes

def pt_in_modes(modes):
    if pd.isna(modes):
        return False
    else:
        return ('BUS' in modes) or ('TRAM' in modes) or ('METRO' in modes)

def bike_in_modes(modes):
    if pd.isna(modes):
        return False
    else:
        return 'BIKE' in modes

def calculate_mode_shares(df_paths, list_id):
    car = 0
    pt = 0
    bike = 0
    no_match = 0
    df_id = df_paths.set_index('ID')
    for id in list_id:
        modes = df_id.loc[id, 'SERVICES']
        if car_in_modes(modes):
            car += 1
        elif pt_in_modes(modes):
            pt += 1
        elif bike_in_modes(modes):
            bike += 1
        else:
            no_match+=1
    sum = len(list_id)
    return (car/sum, pt/sum, bike/sum, no_match/sum)

In [ ]:
list_id = df_path['ID'].unique()

print(calculate_mode_shares(df_path, list_id))

In [ ]:
df_id = df_path.set_index('ID')
for id in list_id:
    modes = df_id.loc[id, 'SERVICES']

In [ ]:
modes

## TTT/TTD

In [ ]:
def calculate_tt_td(list_id, df_users):
    TTT = 0
    TTD = 0
    count = 0
    df_id = df_users.set_index('ID')
    for id in list_id[:10]:
        #df = df_users[df_users['ID']==id]
        df = df_id.loc[id]
        if len(df)>1:
            TTT += Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()
            TTD += df['DISTANCE'].iloc[-1]
            count += 1
    return (TTT/60/count, TTD*1e-3/count)

In [ ]:
# Compute total travel time/distance
list_id = df_users['ID'].unique()
print(calculate_tt_td(list_id, df_users))

In [ ]:
df_id = df_users.set_index('ID')

In [ ]:
df_id#.loc[115712]

In [ ]:
TTT = 0
TTD = 0
count = 0
df_id = df_users.set_index('ID')
for id in list_id[:10000]:
    #df = df_users[df_users['ID']==id]
    df = df_id.loc[id]
    if len(df)>1:
        TTT += Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()
        TTD += df['DISTANCE'].iloc[-1]
        count += 1

In [ ]:
df

In [ ]:
Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()

In [ ]:
df['DISTANCE'].iloc[-1]

In [ ]:
df_path